In [ ]:
!pip install tensorflow
!pip install tensorflow-addons

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
IMAGE_SIZE = (380, 380)
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-4

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],  # Adjust brightness
    horizontal_flip=True,
    validation_split=0.2
)


In [ ]:

train_data = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/kidney_stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/train',  # Path to your CT scan dataset directory
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

Found 963 images belonging to 2 classes.


In [ ]:
validation_data = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/kidney_stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 48 images belonging to 2 classes.


In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE + (3,))
base_model.trainable = False  # Freeze the base model

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Unfreeze the base model after initial training
base_model.trainable = True

# Optionally, freeze the first few layers if needed
for layer in base_model.layers[:150]:  # Adjust the number based on your results
    layer.trainable = False


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)


In [ ]:
model = Model(inputs=base_model.input, outputs=output)


In [ ]:
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True) # Changed .h5 to .keras
]

In [ ]:
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.5078 - loss: 0.6924 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 31s 449ms/step - accuracy: 0.5390 - loss: 0.6916 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 465ms/step - accuracy: 0.4788 - loss: 0.6943 - val_accuracy: 0.5000 - val_loss: 0.6933
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 458ms/step - accuracy: 0.5145 - loss: 0.6922 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 22s 459ms/step - accuracy: 0.4720 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 443ms/step - accuracy: 0.4624 - loss: 0.6958 - val_accuracy: 0.5000 - val_loss: 0.6932


In [ ]:
test_loss, test_accuracy = model.evaluate(validation_data)
print(f"Test Accuracy: {test_accuracy:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 226ms/step - accuracy: 0.4896 - loss: 0.6932
Test Accuracy: 0.50


In [ ]:
model.load_weights('best_model.keras')

In [ ]:
def predict_stone(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    img_array /= 255.0  # Scale pixel values

    prediction = model.predict(img_array)[0][0]
    if prediction > 0.5:
        print("Prediction: Stone present in kidney.")
    else:
        print("Prediction: No stone detected in kidney.")

# Example Usage
predict_stone("/content/drive/MyDrive/kidney_stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal/Normal- (1019).jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: No stone detected in kidney.
